# Assignment: SQL Notebook for Peer Assignment

In [2]:
# Overview of the DataSet
# SpaceX has gained worldwide attention for a series of historic milestones.
# 
# It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010. SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage.
# 
# Therefore if we can determine if the first stage will land, we can determine the cost of a launch.
# 
# This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.
# 
# This dataset includes a record for each payload carried during a SpaceX mission into outer space.

In [3]:
# First load the SQL extension and establish a connection with the database
%load_ext sql

In [4]:
# Import required library
import csv, sqlite3
import prettytable
import pandas as pd
prettytable.DEFAULT = 'DEFAULT'

## Create if not exist or connect the Database

In [5]:
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

## Connect your Jupyter Notebook to a SQLite database

In [6]:
%sql sqlite:///my_data1.db

## Create a data from a csv file loading from online

In [7]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")

## Send to Data from csv to our database 

In [8]:
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [9]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [10]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

# Tasks
Now write and execute SQL queries to solve the assignment tasks.

Note: If the column names are in mixed case enclose it in double quotes For Example "Landing_Outcome"

# Task 1
- Display the names of the unique launch sites in the space mission

In [11]:
%sql SELECT DISTINCT Launch_Site from SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


# Task 2
- Display 5 records where launch sites begin with the string 'CCA'

In [12]:
%sql SELECT DISTINCT Launch_Site from SPACEXTABLE WHERE Launch_Site LIKE '%CCA%';

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
CCAFS SLC-40


# Task 3
- Display the total payload mass carried by boosters launched by NASA (CRS)

In [13]:
%sql SELECT SUM("PAYLOAD_MASS__KG_") AS Total_payload_by_nasa FROM SPACEXTABLE WHERE Customer = 'NASA (CRS)';

 * sqlite:///my_data1.db
Done.


Total_payload_by_nasa
45596


# Task 4
- Display average payload mass carried by booster version F9 v1.1

In [14]:
%sql SELECT AVG("PAYLOAD_MASS__KG_") from SPACEXTABLE WHERE Booster_Version = 'F9 v1.1'

 * sqlite:///my_data1.db
Done.


"AVG(""PAYLOAD_MASS__KG_"")"
2928.4


# Task 5
- List the date when the first succesful landing outcome in ground pad was acheived.

In [15]:
%sql SELECT MIN("Date") as First_Successful_Landing_Ground_Pad from SPACEXTABLE WHERE Landing_Outcome LIKE '%Success%' AND Landing_Outcome LIKE '%ground pad%';

 * sqlite:///my_data1.db
Done.


First_Successful_Landing_Ground_Pad
2015-12-22


# Task 6
- List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [16]:
%sql SELECT "Booster_Version" FROM SPACEXTABLE WHERE "Landing_Outcome" LIKE '%Success (drone ship)%' AND PAYLOAD_MASS__KG_ > 4000 AND PAYLOAD_MASS__KG_ < 6000;

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


# Task 7
- List the total number of successful and failure mission outcomes

In [19]:
%sql SELECT SUM(CASE WHEN "Mission_Outcome" LIKE '%Success%' THEN 1 ELSE 0 END) AS Successful_Missions, SUM(CASE WHEN "Mission_Outcome" LIKE '%Failure%' THEN 1 ELSE 0 END) AS Failed_Missions FROM SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


Successful_Missions,Failed_Missions
100,1


# Task 8
- List all the booster_versions that have carried the maximum payload mass. Use a subquery.

In [20]:
%sql SELECT "Booster_Version" FROM SPACEXTABLE WHERE "PAYLOAD_MASS__KG_" = (SELECT MAX("PAYLOAD_MASS__KG_") FROM SPACEXTABLE);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


# Task 9
- List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.
- Note: SQLLite does not support monthnames. So you need to use substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.

In [22]:
%%sql
SELECT
    CASE substr(Date, 6, 2)
        WHEN '01' THEN 'January'
        WHEN '02' THEN 'February'
        WHEN '03' THEN 'March'
        WHEN '04' THEN 'April'
        WHEN '05' THEN 'May'
        WHEN '06' THEN 'June'
        WHEN '07' THEN 'July'
        WHEN '08' THEN 'August'
        WHEN '09' THEN 'September'
        WHEN '10' THEN 'October'
        WHEN '11' THEN 'November'
        WHEN '12' THEN 'December'
    END AS Month_Name,
    Landing_Outcome,
    Booster_Version,
    Launch_Site
FROM SPACEXTABLE
WHERE Landing_Outcome LIKE '%Failure (drone ship)%'
  AND substr(Date, 1, 4) = '2015';

 * sqlite:///my_data1.db
Done.


Month_Name,Landing_Outcome,Booster_Version,Launch_Site
January,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
April,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


# Task 10
- Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

In [23]:
%%sql
SELECT 
    Landing_Outcome, 
    COUNT(*) AS Outcome_Count
FROM 
    SPACEXTABLE
WHERE 
    Date >= '2010-06-04' AND Date <= '2017-03-20'
GROUP BY 
    Landing_Outcome
ORDER BY 
    Outcome_Count DESC;


 * sqlite:///my_data1.db
Done.


Landing_Outcome,Outcome_Count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
